# Experiment 001: Greedy Baseline with Rotation Optimization

This baseline implements:
1. Greedy tree placement algorithm (from getting-started kernel)
2. Rotation optimization (fix_direction technique)
3. Proper scoring and validation

In [1]:
import math
import random
from decimal import Decimal, getcontext

import numpy as np
import pandas as pd
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import time

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def generate_weighted_angle():
    """
    Generates a random angle with a distribution weighted by abs(sin(2*angle)).
    This helps place more trees in corners, and makes the packing less round.
    """
    while True:
        angle = random.uniform(0, 2 * math.pi)
        if random.uniform(0, 1) < abs(math.sin(2 * angle)):
            return angle

def initialize_trees(num_trees, existing_trees=None):
    """
    Greedy placement algorithm: place trees one by one, starting far from center
    and moving inward until collision.
    """
    if num_trees == 0:
        return [], Decimal('0')

    if existing_trees is None:
        placed_trees = []
    else:
        placed_trees = list(existing_trees)

    num_to_add = num_trees - len(placed_trees)

    if num_to_add > 0:
        unplaced_trees = [
            ChristmasTree(angle=random.uniform(0, 360)) for _ in range(num_to_add)]
        if not placed_trees:
            placed_trees.append(unplaced_trees.pop(0))

        for tree_to_place in unplaced_trees:
            placed_polygons = [p.polygon for p in placed_trees]
            tree_index = STRtree(placed_polygons)

            best_px = None
            best_py = None
            min_radius = Decimal('Infinity')

            # Try 10 random starting attempts and keep the best one
            for _ in range(10):
                angle = generate_weighted_angle()
                vx = Decimal(str(math.cos(angle)))
                vy = Decimal(str(math.sin(angle)))

                radius = Decimal('20.0')
                step_in = Decimal('0.5')

                collision_found = False
                while radius >= 0:
                    px = radius * vx
                    py = radius * vy

                    candidate_poly = affinity.translate(
                        tree_to_place.polygon,
                        xoff=float(px * scale_factor),
                        yoff=float(py * scale_factor))

                    possible_indices = tree_index.query(candidate_poly)
                    if any((candidate_poly.intersects(placed_polygons[i]) and not
                            candidate_poly.touches(placed_polygons[i]))
                           for i in possible_indices):
                        collision_found = True
                        break
                    radius -= step_in

                if collision_found:
                    step_out = Decimal('0.05')
                    while True:
                        radius += step_out
                        px = radius * vx
                        py = radius * vy

                        candidate_poly = affinity.translate(
                            tree_to_place.polygon,
                            xoff=float(px * scale_factor),
                            yoff=float(py * scale_factor))

                        possible_indices = tree_index.query(candidate_poly)
                        if not any((candidate_poly.intersects(placed_polygons[i]) and not
                                   candidate_poly.touches(placed_polygons[i]))
                                   for i in possible_indices):
                            break
                else:
                    radius = Decimal('0')
                    px = Decimal('0')
                    py = Decimal('0')

                if radius < min_radius:
                    min_radius = radius
                    best_px = px
                    best_py = py

            tree_to_place.center_x = best_px
            tree_to_place.center_y = best_py
            tree_to_place.polygon = affinity.translate(
                tree_to_place.polygon,
                xoff=float(tree_to_place.center_x * scale_factor),
                yoff=float(tree_to_place.center_y * scale_factor),
            )
            placed_trees.append(tree_to_place)

    all_polygons = [t.polygon for t in placed_trees]
    bounds = unary_union(all_polygons).bounds

    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor

    width = maxx - minx
    height = maxy - miny
    side_length = max(width, height)

    return placed_trees, side_length

print("Greedy placement functions defined")

Greedy placement functions defined


In [4]:
# Rotation optimization functions (fix_direction technique)

def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side length at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    """Find the optimal rotation angle to minimize bounding box."""
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points) / float(scale_factor)

    hull_points = points_np[ConvexHull(points_np).vertices]

    initial_side = calculate_bbox_side_at_angle(0, hull_points)

    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun

    improvement = initial_side - found_side
    EPSILON = 1e-8

    if improvement > EPSILON:
        best_angle_deg = found_angle_deg
        best_side = Decimal(str(found_side))
    else:
        best_angle_deg = 0.0
        best_side = Decimal(str(initial_side))

    return best_side, best_angle_deg

def apply_rotation(trees, angle_deg):
    """Apply rotation to all trees around the configuration center."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]

    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds) / float(scale_factor)
    min_y = min(b[1] for b in bounds) / float(scale_factor)
    max_x = max(b[2] for b in bounds) / float(scale_factor)
    max_y = max(b[3] for b in bounds) / float(scale_factor)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])

    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])

    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center

    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees

print("Rotation optimization functions defined")

Rotation optimization functions defined


In [5]:
# Validation functions

def has_overlap(trees):
    """Check if any two trees overlap."""
    if len(trees) <= 1:
        return False

    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)

    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def get_side_length(trees):
    """Get the bounding box side length for a configuration."""
    if not trees:
        return 0
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = bounds[0] / float(scale_factor)
    miny = bounds[1] / float(scale_factor)
    maxx = bounds[2] / float(scale_factor)
    maxy = bounds[3] / float(scale_factor)
    return max(maxx - minx, maxy - miny)

def calculate_score(side_lengths):
    """Calculate total score from side lengths."""
    total = 0
    for n, side in side_lengths.items():
        total += (side ** 2) / n
    return total

print("Validation functions defined")

Validation functions defined


In [6]:
# Generate greedy baseline solution
print("Generating greedy baseline solution...")
start_time = time.time()

random.seed(42)  # For reproducibility

all_configurations = {}  # n -> list of trees
side_lengths = {}  # n -> side length

current_placed_trees = []

for n in range(1, 201):
    current_placed_trees, side = initialize_trees(n, existing_trees=current_placed_trees)
    all_configurations[n] = [t.clone() for t in current_placed_trees]
    side_lengths[n] = float(side)
    
    if n % 20 == 0:
        elapsed = time.time() - start_time
        print(f"  N={n}: side={side:.6f}, elapsed={elapsed:.1f}s")

greedy_score = calculate_score(side_lengths)
print(f"\nGreedy baseline score: {greedy_score:.6f}")
print(f"Total time: {time.time() - start_time:.1f}s")

Generating greedy baseline solution...


  N=20: side=4.075499, elapsed=0.3s


  N=40: side=5.881665, elapsed=0.6s


  N=60: side=7.166137, elapsed=1.0s


  N=80: side=8.688082, elapsed=1.4s


...

  N=120: side=10.054136, elapsed=2.6s


  N=140: side=10.948304, elapsed=3.2s


  N=160: side=11.344507, elapsed=4.0s


  N=180: side=12.127983, elapsed=4.8s


  N=200: side=13.125883, elapsed=5.7s

Greedy baseline score: 178.919725
Total time: 5.7s


In [ ]:
# Apply rotation optimization to each configuration
print("\nApplying rotation optimization...")
start_time = time.time()

optimized_configurations = {}
optimized_side_lengths = {}

for n in range(1, 201):
    trees = all_configurations[n]
    
    # Find optimal rotation
    best_side, best_angle = optimize_rotation(trees)
    
    # Apply rotation if it improves
    if best_angle > 0.001:
        rotated_trees = apply_rotation(trees, best_angle)
        new_side = get_side_length(rotated_trees)
        if new_side < side_lengths[n]:
            optimized_configurations[n] = rotated_trees
            optimized_side_lengths[n] = new_side
        else:
            optimized_configurations[n] = trees
            optimized_side_lengths[n] = side_lengths[n]
    else:
        optimized_configurations[n] = trees
        optimized_side_lengths[n] = side_lengths[n]
    
    if n % 50 == 0:
        elapsed = time.time() - start_time
        print(f"  N={n}: original={side_lengths[n]:.6f}, optimized={optimized_side_lengths[n]:.6f}, elapsed={elapsed:.1f}s")

optimized_score = calculate_score(optimized_side_lengths)
print(f"\nOptimized score: {optimized_score:.6f}")
print(f"Improvement: {greedy_score - optimized_score:.6f} ({100*(greedy_score - optimized_score)/greedy_score:.2f}%)")
print(f"Total time: {time.time() - start_time:.1f}s")

In [ ]:
# Validate all configurations
print("\nValidating configurations...")

overlap_count = 0
for n in range(1, 201):
    trees = optimized_configurations[n]
    if has_overlap(trees):
        overlap_count += 1
        print(f"  Overlap detected in N={n}")

if overlap_count == 0:
    print("All configurations are valid (no overlaps)")
else:
    print(f"Found {overlap_count} configurations with overlaps")

In [ ]:
# Create submission file
print("\nCreating submission file...")

index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]

tree_data = []
for n in range(1, 201):
    trees = optimized_configurations[n]
    for tree in trees:
        tree_data.append([float(tree.center_x), float(tree.center_y), float(tree.angle)])

cols = ['x', 'y', 'deg']
submission = pd.DataFrame(index=index, columns=cols, data=tree_data).rename_axis('id')

# Round to 6 decimal places
for col in cols:
    submission[col] = submission[col].astype(float).round(decimals=6)

# Prepend 's' to make values strings
for col in submission.columns:
    submission[col] = 's' + submission[col].astype('string')

# Save submission
submission.to_csv('/home/submission/submission.csv')
print(f"Submission saved to /home/submission/submission.csv")
print(f"Total rows: {len(submission)}")
print(f"\nFinal Score: {optimized_score:.6f}")

In [ ]:
# Final summary
print("="*50)
print("EXPERIMENT SUMMARY")
print("="*50)
print(f"Greedy baseline score: {greedy_score:.6f}")
print(f"After rotation optimization: {optimized_score:.6f}")
print(f"Improvement: {greedy_score - optimized_score:.6f}")
print(f"Target score: 68.922808")
print(f"Gap to target: {optimized_score - 68.922808:.6f}")
print("="*50)